In [1]:
#this notebook runs the PYQG layered model based on the A20 section from 2011, but has been altered to 
#represent the effects of global cooling which have led to decreased stratification and a smaller mixed 
#layer
# http://whp-atlas.ucsd.edu/atlantic/a20/sections/printatlas/printatlas.htm 

In [2]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pyqg

In [3]:
#now to try to run the three layer model(proper PYQG)
from pyqg import diagnostic_tools as tools
#this sets up the layers
L =  1000.e3     # length scale of box    [m]
Ld = 15.e3       # deformation scale      [m]
kd = 1./Ld       # deformation wavenumber [m^-1]
Nx = 256         # number of grid points

H1 = 400.        # layer 1 thickness  [m]
H2 = 1100.       # layer 2
H3 = 3500.       # layer 3

U1 = 0.05          # layer 1 zonal velocity [m/s]
U2 = 0.025         # layer 2
U3 = 0.00          # layer 3

rho1 = 1026.
rho2 = 1027.9
rho3 = 1028.

rek = 1.e-7       # linear bottom drag coeff.  [s^-1]
f0  = 0.0001236812857687059 # coriolis param [s^-1]
beta = 1.2130692965249345e-11 # planetary vorticity gradient [m^-1 s^-1]

Ti = Ld/(abs(U1))  # estimate of most unstable e-folding time scale [s]
dt = Ti/800.   # time-step [s]
tmax = 300*Ti      # simulation time [s]


In [4]:
#this defines the model from all of the set-up parameters above
m = pyqg.LayeredModel(nx=Nx, nz=3, U = [U1,U2,U3],V = [0.,0.,0.],L=L,f=f0,beta=beta,
                         H = [H1,H2,H3], rho=[rho1,rho2,rho3],rek=rek,
                        dt=dt,tmax=tmax, twrite=5000, tavestart=Ti*10)

INFO:  Logger initialized


In [5]:
#this sets up the initial conditions for the model
#in the original code, delta is not specified 
#delta = 0.5
sig = 1.e-7
qi = sig*np.vstack([np.random.randn(m.nx,m.ny)[np.newaxis,],
                    np.random.randn(m.nx,m.ny)[np.newaxis,],
                    np.random.randn(m.nx,m.ny)[np.newaxis,]])
m.set_q(qi)

In [6]:
#now, we run the model 
m.run()

INFO: Step: 5000, Time: 1.88e+06, KE: 3.92e-08, CFL: 0.005
INFO: Step: 10000, Time: 3.75e+06, KE: 8.41e-08, CFL: 0.005
INFO: Step: 15000, Time: 5.62e+06, KE: 3.25e-07, CFL: 0.005
INFO: Step: 20000, Time: 7.50e+06, KE: 1.70e-06, CFL: 0.005
INFO: Step: 25000, Time: 9.38e+06, KE: 9.93e-06, CFL: 0.005
INFO: Step: 30000, Time: 1.12e+07, KE: 6.07e-05, CFL: 0.006
INFO: Step: 35000, Time: 1.31e+07, KE: 3.58e-04, CFL: 0.014
INFO: Step: 40000, Time: 1.50e+07, KE: 1.73e-03, CFL: 0.029
INFO: Step: 45000, Time: 1.69e+07, KE: 3.92e-03, CFL: 0.037
INFO: Step: 50000, Time: 1.88e+07, KE: 7.72e-03, CFL: 0.059
INFO: Step: 55000, Time: 2.06e+07, KE: 1.58e-02, CFL: 0.070
INFO: Step: 60000, Time: 2.25e+07, KE: 2.60e-02, CFL: 0.080
INFO: Step: 65000, Time: 2.44e+07, KE: 3.76e-02, CFL: 0.104
INFO: Step: 70000, Time: 2.62e+07, KE: 5.75e-02, CFL: 0.117
INFO: Step: 75000, Time: 2.81e+07, KE: 1.00e-01, CFL: 0.140
INFO: Step: 80000, Time: 3.00e+07, KE: 1.45e-01, CFL: 0.175
INFO: Step: 85000, Time: 3.19e+07, KE: 2.

AssertionError: None

In [ ]:
#now this visualizes the results
plt.figure(figsize=(18,4))

plt.subplot(131)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,(m.q[0,]+m.Qy[0]*m.y)/(U1/Ld),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('Layer 1 PV')

plt.subplot(132)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,(m.q[1,]+m.Qy[1]*m.y)/(U1/Ld),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('Layer 2 PV')

plt.subplot(133)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,(m.q[2,]+m.Qy[2]*m.y)/(U1/Ld),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('Layer 3 PV')

In [ ]:
kr, kespec_1 = tools.calc_ispec(m,m.get_diagnostic('KEspec')[0])
_, kespec_2 = tools.calc_ispec(m,m.get_diagnostic('KEspec')[1])
_, kespec_3 = tools.calc_ispec(m,m.get_diagnostic('KEspec')[2])

plt.loglog( kr, kespec_1, '.-' )
plt.loglog( kr, kespec_2, '.-' )
plt.loglog( kr, kespec_3, '.-' )

plt.legend(['layer 1','layer 2', 'layer 3'], loc='lower left')
plt.ylim([1e-14,1e-6]); plt.xlim([m.kk.min(), m.kk.max()])
plt.xlabel(r'k (m$^{-1}$)'); plt.grid()
plt.title('Scenario 2 Kinetic Energy Spectrum');